## Radio Frequency (RF) Signal Image Classification

#### Dataset
- The [dataset](https://www.kaggle.com/datasets/halcy0nic/radio-frequecy-rf-signal-image-classification/data) contains images of [waterfall plots](https://en.wikipedia.org/wiki/Waterfall_plot), which were generated by spectrum analyzer.
- Waterfall plot is a type of visualization used to display frequency content of a signal over time. The color in a waterfall plot represents the signal strength at a particular frequency and time. The specific color mapping used can vary, but typically, warmer colors (like red and orange) indicate higher signal strength, while cooler colors (like blue and green) indicate lower signal strength.

In [1]:
# Install CUDA-Enabled PyTorch - pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA RTX A4000


In [ ]:
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import os
from minio import Minio
from io import BytesIO
from PIL import Image
from dotenv import load_dotenv
from torchvision.models import ResNet18_Weights

# Load environment variables
load_dotenv()

True

In [3]:
# Set Seeds for Reproducibility
seed = 30
torch.manual_seed(seed) # Set the seed for generating random numbers on the CPU. By setting this seed, you ensure that the random numbers generated by PyTorch on the CPU are reproducible.
torch.cuda.manual_seed(seed) # Set the seed for generating random numbers on the GPU. By setting this seed, you ensure that the random numbers generated by PyTorch on the GPU are reproducible.
torch.backends.cudnn.deterministic = True # Ensure that the CUDA backend (cuDNN) uses deterministic algorithms. This is important for reproducibility, as some algorithms in cuDNN can have non-deterministic behavior.

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
# MinIO Configuration
minio_client = Minio(
    os.getenv("MINIO_IP"),
    access_key=os.getenv("MINIO_ACCESS_KEY"),
    secret_key=os.getenv("MINIO_SECRET_KEY"),
    secure=False
)

bucket_name = os.getenv("MINIO_BUCKET_NAME")
prefix = os.getenv("PREFIX")
local_tmp_dir = os.getenv("LOCAL_TMP_DIR")
os.makedirs(local_tmp_dir, exist_ok=True)

# Download images from MinIO
def download_images_from_minio():
    objects = minio_client.list_objects(bucket_name, prefix=prefix, recursive=True)
    for obj in objects:
        if obj.object_name.endswith(('.jpg', '.png', '.jpeg')):
            response = minio_client.get_object(bucket_name, obj.object_name)
            
            # Create subdirectories to match MinIO structure
            relative_path = obj.object_name[len(prefix):]  # Remove prefix
            class_folder = os.path.join(local_tmp_dir, os.path.dirname(relative_path))
            os.makedirs(class_folder, exist_ok=True)
            
            img_path = os.path.join(local_tmp_dir, relative_path)
            with open(img_path, 'wb') as file_data:
                file_data.write(response.read())

download_images_from_minio()

In [6]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images for CNN
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset
dataset = datasets.ImageFolder(root=local_tmp_dir, transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define CNN Model
class RFSignalCNN(nn.Module):
    def __init__(self, num_classes):
        super(RFSignalCNN, self).__init__()
        # self.model = models.resnet18(pretrained=True)
        self.model = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.model(x)

# Initialize model
num_classes = len(dataset.classes)
model = RFSignalCNN(num_classes).to(device)

c:\Python\Keysight\TSM\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python\Keysight\TSM\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\desktop-4ce407b4p2/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 83.8MB/s]


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

train_model(model, train_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Loss: 0.8640
Epoch 2/10, Loss: 0.3926
Epoch 3/10, Loss: 0.2153
Epoch 4/10, Loss: 0.1729
Epoch 5/10, Loss: 0.1340
Epoch 6/10, Loss: 0.1251
Epoch 7/10, Loss: 0.1008
Epoch 8/10, Loss: 0.0359
Epoch 9/10, Loss: 0.0739
Epoch 10/10, Loss: 0.0847


In [12]:
# Save the model's state dict
# torch.save(model.state_dict(), 'trained_model.pth')

# To load the model
# model = YourModelClass()  # Make sure to define the same model structure
model.load_state_dict(torch.load('trained_model.pth'))
model.eval()  # Set to evaluation mode if necessary

RFSignalCNN(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_r

In [9]:
# Evaluate model
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate_model(model, test_loader)

Test Accuracy: 95.17%


In [14]:
# Predict a new image
def predict_image(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return class_names[predicted.item()]

# Example usage:
new_image_path = "minio_data/wifi/865694df4a47b2e8edef4aecf5196da1.png"
predicted_label = predict_image(new_image_path, model, transform, dataset.classes)
print(f"Predicted label: {predicted_label}")


Predicted label: wifi
